In [1]:
import pandas as pd
import re
import xmltodict
import flatdict
from pathlib import Path
import os
from edgar import run_form4, form4_to_csv, Form4Data

In [2]:
text =r"""<TEXT>
<XML>
<?xml version="1.0"?>
<ownershipDocument>
    <issuer>
        <issuerCik>0000912728</issuerCik>
        <issuerName>FORWARD AIR CORP</issuerName>
        <issuerTradingSymbol>FWRD</issuerTradingSymbol>
    </issuer>

    <nonDerivativeTable>
        <nonDerivativeTransaction>
            <securityTitle>
                <value>Common Stock</value>
            </securityTitle>
            <transactionDate>
                <value>2020-11-27</value>
            </transactionDate>
            <transactionCoding>
                <transactionFormType>4</transactionFormType>
                <transactionCode>M</transactionCode>
                <equitySwapInvolved>0</equitySwapInvolved>
            </transactionCoding>
            <transactionAmounts>
                <transactionShares>
                    <value>3033</value>
                </transactionShares>
                <transactionPricePerShare>
                    <value>47.82</value>
                </transactionPricePerShare>
                <transactionAcquiredDisposedCode>
                    <value>A</value>
                </transactionAcquiredDisposedCode>
            </transactionAmounts>

        </nonDerivativeTransaction>
    </nonDerivativeTable>
</ownershipDocument>
</XML>
</TEXT>
"""

In [3]:
matcher = re.compile(r'<\?xml.*ownershipDocument>', flags=re.DOTALL)
matches = matcher.search(text)
xml     = matches.group(0)
print(xml)

<?xml version="1.0"?>
<ownershipDocument>
    <issuer>
        <issuerCik>0000912728</issuerCik>
        <issuerName>FORWARD AIR CORP</issuerName>
        <issuerTradingSymbol>FWRD</issuerTradingSymbol>
    </issuer>

    <nonDerivativeTable>
        <nonDerivativeTransaction>
            <securityTitle>
                <value>Common Stock</value>
            </securityTitle>
            <transactionDate>
                <value>2020-11-27</value>
            </transactionDate>
            <transactionCoding>
                <transactionFormType>4</transactionFormType>
                <transactionCode>M</transactionCode>
                <equitySwapInvolved>0</equitySwapInvolved>
            </transactionCoding>
            <transactionAmounts>
                <transactionShares>
                    <value>3033</value>
                </transactionShares>
                <transactionPricePerShare>
                    <value>47.82</value>
                </transactionPricePerShare>
      

In [4]:
xmldict = xmltodict.parse(xml)
print(xmldict)

{'ownershipDocument': {'issuer': {'issuerCik': '0000912728', 'issuerName': 'FORWARD AIR CORP', 'issuerTradingSymbol': 'FWRD'}, 'nonDerivativeTable': {'nonDerivativeTransaction': {'securityTitle': {'value': 'Common Stock'}, 'transactionDate': {'value': '2020-11-27'}, 'transactionCoding': {'transactionFormType': '4', 'transactionCode': 'M', 'equitySwapInvolved': '0'}, 'transactionAmounts': {'transactionShares': {'value': '3033'}, 'transactionPricePerShare': {'value': '47.82'}, 'transactionAcquiredDisposedCode': {'value': 'A'}}}}}}


In [5]:
print(flatdict.FlatDict(xmldict["ownershipDocument"], delimiter='.'))

{'issuer.issuerCik': '0000912728', 'issuer.issuerName': 'FORWARD AIR CORP', 'issuer.issuerTradingSymbol': 'FWRD', 'nonDerivativeTable.nonDerivativeTransaction.securityTitle.value': 'Common Stock', 'nonDerivativeTable.nonDerivativeTransaction.transactionDate.value': '2020-11-27', 'nonDerivativeTable.nonDerivativeTransaction.transactionCoding.transactionFormType': '4', 'nonDerivativeTable.nonDerivativeTransaction.transactionCoding.transactionCode': 'M', 'nonDerivativeTable.nonDerivativeTransaction.transactionCoding.equitySwapInvolved': '0', 'nonDerivativeTable.nonDerivativeTransaction.transactionAmounts.transactionShares.value': '3033', 'nonDerivativeTable.nonDerivativeTransaction.transactionAmounts.transactionPricePerShare.value': '47.82', 'nonDerivativeTable.nonDerivativeTransaction.transactionAmounts.transactionAcquiredDisposedCode.value': 'A'}


In [6]:
# run test with specific txt file
input_path = Path("./tests/test_jup")
output_path = Path("./tests/test_jup/scr")
output_path.mkdir(exist_ok=True)

nonDerivative_file = output_path / "nonDerivative.csv"
derivative_file = output_path / "derivative.csv"

filename ="912728_4_0000912728-20-000168.txt"
filename ="1109354_3_0001179110-20-009902.txt"

nonDerivative_file.unlink(missing_ok=True)
derivative_file.unlink(missing_ok=True)
form4_to_csv(input_path, output_path, filename)


In [7]:
# run 100 test
# nonDerivative_file.unlink(missing_ok=True)
# derivative_file.unlink(missing_ok=True)
# run_form4(input_path, output_path, True)

In [8]:
# nd = pd.read_csv(nonDerivative_file)
# d = pd.read_csv(derivative_file)
# display(nd)


In [9]:
nd = Form4Data.from_csv(nonDerivative_file)
# d = Form4Data.from_csv(derivative_file)
display(nd.df)
nd.add_has_10b5()
display(nd.df)
display(nd.df.loc[nd.df["has_10b5"]])

,issuerCik,issuerName,issuerTradingSymbol,reportingOwnerId.rptOwnerCik,reportingOwnerId.rptOwnerName,reportingOwnerAddress.rptOwnerStreet1,reportingOwnerAddress.rptOwnerStreet2,reportingOwnerAddress.rptOwnerCity,reportingOwnerAddress.rptOwnerState,reportingOwnerAddress.rptOwnerZipCode,...,deemedExecutionDate.value,transactionCoding.transactionCode,transactionTimeliness.value,transactionAmounts.transactionShares.value,transactionAmounts.transactionAcquiredDisposedCode.value,transactionAmounts.transactionPricePerShare.value,postTransactionAmounts.sharesOwnedFollowingTransaction.value,ownershipNature.directOrIndirectOwnership.value,ownershipNature.natureOfOwnership.value,footnote
0,1109354,BRUKER CORP,BRKR,1675108,Friend Cynthia M,BRUKER CORPORATION,40 MANNING ROAD,BILLERICA,MA,1821,...,NaN,M,NaN,800,A,26.1700,12657,D,NaN,NaN
1,1109354,BRUKER CORP,BRKR,1675108,Friend Cynthia M,BRUKER CORPORATION,40 MANNING ROAD,BILLERICA,MA,1821,...,NaN,S,NaN,800,D,37.5813,11857,D,NaN,F2: The price reported in Column 4 is a weight...


,issuerCik,issuerName,issuerTradingSymbol,reportingOwnerId.rptOwnerCik,reportingOwnerId.rptOwnerName,reportingOwnerAddress.rptOwnerStreet1,reportingOwnerAddress.rptOwnerStreet2,reportingOwnerAddress.rptOwnerCity,reportingOwnerAddress.rptOwnerState,reportingOwnerAddress.rptOwnerZipCode,...,transactionCoding.transactionCode,transactionTimeliness.value,transactionAmounts.transactionShares.value,transactionAmounts.transactionAcquiredDisposedCode.value,transactionAmounts.transactionPricePerShare.value,postTransactionAmounts.sharesOwnedFollowingTransaction.value,ownershipNature.directOrIndirectOwnership.value,ownershipNature.natureOfOwnership.value,footnote,has_10b5
0,1109354,BRUKER CORP,BRKR,1675108,Friend Cynthia M,BRUKER CORPORATION,40 MANNING ROAD,BILLERICA,MA,1821,...,M,NaN,800,A,26.1700,12657,D,NaN,NaN,False
1,1109354,BRUKER CORP,BRKR,1675108,Friend Cynthia M,BRUKER CORPORATION,40 MANNING ROAD,BILLERICA,MA,1821,...,S,NaN,800,D,37.5813,11857,D,NaN,F2: The price reported in Column 4 is a weight...,True


,issuerCik,issuerName,issuerTradingSymbol,reportingOwnerId.rptOwnerCik,reportingOwnerId.rptOwnerName,reportingOwnerAddress.rptOwnerStreet1,reportingOwnerAddress.rptOwnerStreet2,reportingOwnerAddress.rptOwnerCity,reportingOwnerAddress.rptOwnerState,reportingOwnerAddress.rptOwnerZipCode,...,transactionCoding.transactionCode,transactionTimeliness.value,transactionAmounts.transactionShares.value,transactionAmounts.transactionAcquiredDisposedCode.value,transactionAmounts.transactionPricePerShare.value,postTransactionAmounts.sharesOwnedFollowingTransaction.value,ownershipNature.directOrIndirectOwnership.value,ownershipNature.natureOfOwnership.value,footnote,has_10b5
1,1109354,BRUKER CORP,BRKR,1675108,Friend Cynthia M,BRUKER CORPORATION,40 MANNING ROAD,BILLERICA,MA,1821,...,S,NaN,800,D,37.5813,11857,D,NaN,F2: The price reported in Column 4 is a weight...,True


In [10]:
d = Form4Data.from_csv(derivative_file)
display(d.df)

,issuerCik,issuerName,issuerTradingSymbol,reportingOwnerId.rptOwnerCik,reportingOwnerId.rptOwnerName,reportingOwnerAddress.rptOwnerStreet1,reportingOwnerAddress.rptOwnerStreet2,reportingOwnerAddress.rptOwnerCity,reportingOwnerAddress.rptOwnerState,reportingOwnerAddress.rptOwnerZipCode,...,transactionAmounts.transactionShares.value,exerciseDate.value,expirationDate.value,underlyingSecurity.underlyingSecurityTitle.value,underlyingSecurity.underlyingSecurityShares.value,transactionAmounts.transactionPricePerShare.value,postTransactionAmounts.sharesOwnedFollowingTransaction.value,ownershipNature.directOrIndirectOwnership.value,ownershipNature.natureOfOwnership.value,footnote
0,1109354,BRUKER CORP,BRKR,1675108,Friend Cynthia M,BRUKER CORPORATION,40 MANNING ROAD,BILLERICA,MA,1821,...,800,NaN,2026-05-20,Common Stock,800,0,0,D,NaN,F3: The option vested in three approximately e...
